In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, AdamW, AutoModel


In [1]:
def read_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = [line.strip() for line in file.readlines()]
    return sentences

file_path_with_diacritics = 'sentences_with_diacritics.txt'
file_path_without_diacritics = 'sentences_without_diacritics.txt'

list_of_sentences_with_diacritics = read_sentences(file_path_with_diacritics)
list_of_sentences_without_diacritics = read_sentences(file_path_without_diacritics)

print("Sentences with diacritics:", list_of_sentences_with_diacritics[:5])
print("Sentences without diacritics:", list_of_sentences_without_diacritics[:5])


Sentences with diacritics: ['قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ هُوَ كَالزِّنْدِيقِ إذَا عَمِلَ السِّحْرَ بِنَفْسِهِ قُتِلَ وَلَمْ يُسْتَتَبْ', 'قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ أَيْ الْوَصِيَّةُ قَوْلُهُ مَا مَرَّ أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا لَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَم

In [3]:
model_name="asafaya/bert-base-arabic"
tokenizer= AutoTokenizer.from_pretrained(model_name)
model= AutoModel.from_pretrained(model_name)

In [ ]:

def get_embeddings(text):
    tokens= tokenizer(text,return_tensors="pt", truncation=True, max_length=512)
    model_output= model(**tokens)

    return model_output.last_hidden_state.detach().numpy()

auto_sentences_embeddings=[get_embeddings(sentence) for sentence in list_of_string_sentences]

print(len(auto_sentences_embeddings))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
model = AutoModel.from_pretrained("asafaya/bert-base-arabic")

def get_embeddings(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    model_output = model(**tokens)

    return model_output.last_hidden_state.detach().numpy()

# Assuming list_of_string_sentences is a list of undiacritized sentences
undiacritized_sentences = list_of_string_sentences

# Create a list of tuples, where each tuple contains an undiacritized sentence and its diacritized version
paired_sentences = [(undiacritized_sentence, diacritized_sentence) for undiacritized_sentence, diacritized_sentence in zip(undiacritized_sentences, diacritized_sentences)]

# Get embeddings for each paired sentence
paired_sentences_embeddings = [(get_embeddings(undiacritized_sentence), diacritized_sentence) for undiacritized_sentence, diacritized_sentence in paired_sentences]

print(len(paired_sentences_embeddings))


In [ ]:

# Sample data (replace with your actual data)
train_data = [("سلام", "سَلام"), ("مرحبا", "مَرحبًا"), ...]
dev_data = [("مرحبا", "مَرحبًا"), ...]
test_data = [("أهلا", ""), ...]

# Load pre-trained model and tokenizer
model_name = "asafaya/bert-base-arabic"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(tokenizer))

# Tokenize and encode data
train_encodings = tokenizer(train_data, return_tensors="pt", padding="max_length", truncation=True)
dev_encodings = tokenizer(dev_data, return_tensors="pt", padding="max_length", truncation=True)
test_encodings = tokenizer(test_data, return_tensors="pt", padding="max_length", truncation=True)

# Extract labels from encoded data
train_labels = torch.tensor([[label_ids] for label_ids in tokenizer(train_data[1], return_tensors="pt")["input_ids"]])
dev_labels = torch.tensor([[label_ids] for label_ids in tokenizer(dev_data[1], return_tensors="pt")["input_ids"]])

# Fine-tune the model
optimizer = AdamW(model.parameters(), lr=2e-5)
for epoch in range(3):
    optimizer.zero_grad()
    outputs = model(**train_encodings, labels=train_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Evaluate on dev set
with torch.no_grad():
    outputs = model(**dev_encodings, labels=dev_labels)
    # Process outputs for evaluation

# Test the model on the test set
with torch.no_grad():
    test_outputs = model(**test_encodings)
    # Process test_outputs for testing
